<a href="https://colab.research.google.com/github/slipaway/melonplaylist/blob/master/tag2vec%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전처리 하기 위한 준비

In [ ]:
#형태소 분석기
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

In [5]:
#구글 계정 마운트
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from datetime import timedelta, datetime
import glob
import os
import re
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import re
from collections import Counter
from typing import *
from khaiii import KhaiiiApi

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
folder = '/content/gdrive/My Drive/멜론 플레이리스트' #working directory
filelist = os.chdir(folder)

In [8]:
train=pd.read_json('train.json')
test = pd.read_json('test.json')

In [ ]:
!git clone https://github.com/slipaway/melonplaylist.git

# 0. 함수 정의

In [7]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

def uppertolower(string):
  if isEnglish(string) == True:
    string = string.lower() #태그나 제목이 영어로 들어간 경우, 모두 소문자로 바꿔주기

In [73]:
def re_sub(series: pd.Series) -> pd.Series:
  series = series.str.replace(pat=r'[ㄱ-ㅎ]', repl=r'', regex=True) #ㅋ 제거용
  series = series.str.replace(pat=r'[^\w\s]', repl=r'', regex=True) #특수문자 제거용
  series = series.str.replace(pat=r'[ ]{2,}', repl=r' ', regex=True) #공백 제거
  series = series.str.replace(pat=r'[\u3000]+', repl=r'', regex=True) #u3000제거
  return series

def flatten(list_of_list : List) -> List:
  flatten = [j for i in list_of_list for j in i]
  return flatten

def get_token(title: str, tokenizer)-> List[Tuple]:

  if len(title)==0 or title== ' ' : #제목이 공백인 경우 tokenizer 에러 발생
    return []
  
  result = tokenizer.analyze(title)
  result = [(morph.lex, morph.tag) for split in result for morph in split.morphs] # (형태소, 품사) 튜플의 리스트
  return result

def get_all_tags(df) -> List:
  tag_list = df['tags'].values.tolist()
  tag_list=flatten(tag_list)
  return tag_list

# 1. 태그 전처리

In [ ]:
 train['tags'].head(10)

In [ ]:
#영어는 소문자로 모두 바꾸기
for i in range(len(train['tags'])):
  for j in range(len(train['tags'][i])):
    train['tags'][i][j]= train['tags'][i][j].lower()

In [85]:
tokenizer = KhaiiiApi()
all_tag = get_all_tags(train)
token_tag = [get_token(x, tokenizer) for x in all_tag] #태그를 형태소 분석

In [ ]:
token_tag[:40]

In [90]:
token_itself=list(filter(lambda x: len(x)==1, token_tag)) # 태그 자체가 형태소여서 분리되지 않는 태그만 고름
token_itself=flatten(token_itself)
flatten_token=flatten(token_tag)

In [91]:
print('%-23s'%'# of original tag is', f'{len(all_tag):8,}')
print('%-23s'%'# of morpheme itself is', f'{len(token_itself):8,}')
print('%-23s'%'# of total token is', f'{len(flatten_token):8,}')

# of original tag is     476,331
# of morpheme itself is  333,093
# of total token is      722,860


In [92]:
#counting part of speech (pos)
pos=[x[1] for x in token_itself]
pos_count=Counter(pos)
popular_pos = pos_count.most_common()

In [ ]:
#tag 분류표
objects=[x[0] for x in popular_pos]
y_pos=np.arange(len(objects))
performance=[x[1] for x in popular_pos]

plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('Usage')
plt.title('Part of Speech - Tags')
plt.show()

# 2. 태그: word2vec 임베딩 with train dataset



In [9]:
from gensim.test.utils import get_tmpfile
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models import Word2Vec

class EpochSaver(CallbackAny2Vec):
  def __init__(self, path_prefix):
    self.path_prefix = path_prefix
    self.epoch = 0

  def on_epoch_end(self, model):
    output_path = get_tmpfile('{}_epoch{}.model'.format(self.path_prefix, self.epoch))
    model.save(output_path)
    self.epoch += 1

class EpochLogger(CallbackAny2Vec):
     def __init__(self):
         self.epoch = 0

     def on_epoch_begin(self, model):
         print("Epoch #{} start".format(self.epoch))

     def on_epoch_end(self, model):
         print("Epoch #{} end".format(self.epoch))
         self.epoch += 1

In [10]:
tag_list = [train['tags'][i] for i in range(len(train['tags']))]
epoch_logger = EpochLogger()
tag_embedding_model = Word2Vec(tag_list, window = 3, min_count=3, workers=4, iter=1000, sg=1, callbacks=[epoch_logger])

Epoch #0 start
Epoch #0 end
Epoch #1 start
Epoch #1 end
Epoch #2 start
Epoch #2 end
Epoch #3 start
Epoch #3 end
Epoch #4 start
Epoch #4 end
Epoch #5 start
Epoch #5 end
Epoch #6 start
Epoch #6 end
Epoch #7 start
Epoch #7 end
Epoch #8 start
Epoch #8 end
Epoch #9 start
Epoch #9 end
Epoch #10 start
Epoch #10 end
Epoch #11 start
Epoch #11 end
Epoch #12 start
Epoch #12 end
Epoch #13 start
Epoch #13 end
Epoch #14 start
Epoch #14 end
Epoch #15 start
Epoch #15 end
Epoch #16 start
Epoch #16 end
Epoch #17 start
Epoch #17 end
Epoch #18 start
Epoch #18 end
Epoch #19 start
Epoch #19 end
Epoch #20 start
Epoch #20 end
Epoch #21 start
Epoch #21 end
Epoch #22 start
Epoch #22 end
Epoch #23 start
Epoch #23 end
Epoch #24 start
Epoch #24 end
Epoch #25 start
Epoch #25 end
Epoch #26 start
Epoch #26 end
Epoch #27 start
Epoch #27 end
Epoch #28 start
Epoch #28 end
Epoch #29 start
Epoch #29 end
Epoch #30 start
Epoch #30 end
Epoch #31 start
Epoch #31 end
Epoch #32 start
Epoch #32 end
Epoch #33 start
Epoch #33 end


In [11]:
list(tag_embedding_model.wv.vocab.keys())

['락',
 '추억',
 '회상',
 '까페',
 '잔잔한',
 '연말',
 '눈오는날',
 '캐럴',
 '분위기',
 '따듯한',
 '크리스마스캐럴',
 '겨울노래',
 '크리스마스',
 '겨울왕국',
 '크리스마스송',
 '댄스',
 '운동',
 '드라이브',
 'Pop',
 '트로피컬하우스',
 '힐링',
 '기분전환',
 '2017',
 '팝',
 '트렌드',
 '일렉',
 '짝사랑',
 '취향저격',
 '슬픔',
 '고백',
 '사랑',
 '이별',
 '일렉트로니카',
 '포크',
 '메탈',
 '인디',
 '록',
 'Metal',
 'M에센셜',
 'Rock',
 'kpop',
 '걸그룹댄스',
 '스트레스해소',
 '새해',
 '여행',
 '프로필음악',
 '카카오톡',
 '소원',
 '프로필',
 '소망',
 '다짐',
 '카톡',
 '듣고',
 '우울',
 '힘내',
 '힙합',
 '느낌있는',
 '밤',
 '새벽',
 'RnB',
 '감각적인',
 '국내',
 '그루브한',
 '가을',
 '재즈',
 '감성',
 '질리지않는',
 '나만알고싶은',
 '봄',
 '설렘',
 '비오는날',
 '스밍',
 '목록',
 '폐막식',
 '올림픽',
 '엑소',
 '조용히',
 '혼자',
 '새벽감성',
 '고민',
 '맥주한잔',
 '카페',
 'OST',
 '어쿠스틱',
 '편안한',
 '에너지',
 '듀엣',
 '달달',
 '피쳐링',
 '남녀',
 '신인',
 '국힙',
 '신나는',
 '존윌리엄스',
 '영화클래식',
 '겨울',
 '아침',
 '빅밴드',
 '상쾌한',
 '밝은',
 '스윙',
 '스윙재즈',
 '휴식',
 '여름',
 '매장음악',
 '헬스',
 '스포츠',
 '피트니스',
 '다이어트',
 '런닝',
 '필라테스',
 '산책',
 '요가',
 '발라드',
 '캐롤',
 '위로',
 '80년대',
 '90년대발라드',
 '올드송',
 '90년대',
 '옛날노래',
 '월드',
 '연주',
 '클럽',
 '스트레스',
 '주

예시.

In [ ]:
tag_embedding_model.wv.most_similar('연말')

In [ ]:
tag_embedding_model.wv.most_similar('조용')

In [ ]:
tag_embedding_model.wv.most_similar(positive = ['감성','기분전환'])

# 3. 태그 예측하기

# Test: tag 있는 것들 중에서 tag 예측

In [202]:
#태그가 있는 플레이리스트 인덱스 뽑아내기
tag_yu_index = []
tag_no_index = []
for i in range(len(test['tags'])):
  if len(test['tags'][i]) != 0:
    tag_yu_index.append(i)
  else:
    tag_no_index.append(i)

In [203]:
test_tag_yu = test.loc[tag_yu_index,:]
test_tag_yu = test_tag_yu.reset_index()

In [204]:
test_tag_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date
0,1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000
1,2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000
2,3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000
3,5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000
4,7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000
...,...,...,...,...,...,...,...
5417,10733,[별],77688,,"[93507, 324019, 221377, 656426, 26083, 661209,...",8,2018-08-10 20:14:19.000
5418,10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000
5419,10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000
5420,10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000


In [154]:
test_tag_list = [list(filter(lambda x: x in list(tag_embedding_model.wv.vocab.keys()), test_tag_yu['tags'][i])) for i in range(len(test_tag_yu['tags']))]

In [155]:
list_for_list = []
val_predicted_tag = []
for i in range(len(test_tag_list)):
  if len(test_tag_list[i]) == 0:
    val_predicted_tag.append([])
  else:
    for k in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive = test_tag_list[i])[k][0])
    val_predicted_tag.append(list_for_list)
    print(i)
    list_for_list = []

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
295
296
297
298
300
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
411
412
413
414
415
416
417
418
419
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519


In [156]:
test_tag_yu['predicted_tags']=val_predicted_tag
test_tag_yu

,index,tags,id,plylst_title,songs,like_cnt,updt_date,predicted_tags
0,1,"[나만의Best3, 인디아티스트들의추천음악]",7461,,"[196298, 269984, 267805, 175867, 529244, 63825...",0,2019-12-17 14:06:45.000,"[피쉬슈즈, 피터팬컴플렉스, 소수빈, 인디스트리트, 케이인디차트, yasisi, 미..."
1,2,[드라이브],90348,,"[273433, 331003, 68432, 411659, 117793, 616860...",21,2015-05-23 10:44:48.000,"[기분전환, 여행, 빵빵, 운전, 신나는, 흥, 신나는드라이브, 휴게소, 운전할때,..."
2,3,[분위기],58617,,"[702227, 48152, 440008, 358488, 701041, 540721...",0,2019-03-14 09:47:34.000,"[분위기있는, 어두움, 레트로감성, 비밥, 관능적, 매혹, 감성, 분위기를, 패션위..."
3,5,[스트레스],88862,노래방 안에서,[],22,2016-01-22 20:55:12.000,"[내적댄스, 기분전환, 시원시원, 신나는노래모음, 프로야구, 드라아브, 국내EDM,..."
4,7,[사랑],133975,봄바람과 듣고싶은 설레는 노래,[],0,2020-02-19 03:05:49.000,"[진선미, 반함, 달달송, 상사병, 첫만남, 프로포즈송, 생각했던대로, 남여, 설렌..."
...,...,...,...,...,...,...,...,...
5417,10733,[별],77688,,"[93507, 324019, 221377, 656426, 26083, 661209,...",8,2018-08-10 20:14:19.000,"[달빛, 밤하늘, 선선한바람, 여름밤, 가을밤, 달, 달밤, 무중력, 여수밤바다, 밤]"
5418,10735,[추억],137930,,"[323755, 397594, 445908, 570242, 221853, 20018...",16,2016-04-18 11:02:09.000,"[회상, 이별은, 담시, 슈가맨시즌2, 올드감성, 재발견, 토토가, 첫이별, 토토가..."
5419,10736,"[띵곡의, 우울, 분위기, 드라이브, 산책]",936,,"[105140, 582252, 199262, 422915, 547967, 48791...",1,2020-04-08 07:15:59.000,"[산책해요, 아쿠스틱, 우울하고, 띵_곡, 오후의, 공감하는, 혼자, 생각을, 운전..."
5420,10737,[기분전환],110589,,"[21976, 207746, 40025, 31635, 567462, 641799, ...",6,2016-06-29 00:57:21.000,"[드라이브, 신나는, 여행, 핫한노래, 스트레스, 제주여행, 팝, 청량한, 내적댄스..."


In [158]:
a = test_tag_yu['id']
b = test_tag_yu['predicted_tags']
c = pd.concat([a, b], axis=1)
c.to_json('H_result_1.json') #json 파일로 저장

In [ ]:
tag2vec_result = c
tag2vec

# 4. tag2vec에서 비어있는 데이터 처리하기

tag2vec에서 결과를 출력하는데 실패한 행(row)이 있다. 태그를 뽑지 못한 이유로는 "그 행에 있는 태그 모두가 train 과정에서 학습되지 못한 태그였기 때문"인 것으로 보인다. 이 데이터의 경우, MF 모델을 사용하여 태그를 메우기로 결정하였다. MF 모델에서 태그를 출력하기 위해 tag2vec 결과물이 없는 플레이리스트 id를 뽑아내었다. (이후 결과는 MF 모델 참고)

In [ ]:
no_tag2vec_index = []
for i in range(len(tag2vec['predicted_tags'])):
  if len(tag2vec['predicted_tags'][i]) == 0:
    no_tag2vec_index.append(tag2vec['id'][i])

In [ ]:
no_tag2vec_index.to_csv('no_tag2vec_index.csv')

# 5. 다른 모델에서 부족한 데이터 채우기

train2vec 모델에서 나온 결과물 중 태그 개수가 10개가 되지 않는 것들이 있었다. 우리가 예측해야 할 태그의 개수는 총 10개여야 하므로, tag2vec 모델을 추가적으로 사용하여 부족한 태그의 개수를 메어주었다. (train2vec 모델에서 나온 태그를 tag2vec 모델에 입력한 뒤, 부족한 개수만큼 tag2vec이 보충해주는 방식)

In [180]:
tag_addition = pd.read_json('곡만 존재하는 플레이리스트에서 predicted_tags 출력.json')
tag_addition

,id,ply_token,tags,songs,predicted_songs,predicted_tags
0,70107,[],[],"[398985, 449403, 411543, 528044, 143048, 98020...",[],"[황현, 프리스타일, Apink, 고등학생]"
1,102395,[],[],"[528550, 285114, 506667, 339507, 653012, 435029]",[],"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,[],[],"[538867, 513110, 225168, 211527, 425074, 21709...",[],"[weeflow, kxan]"
3,13244,[],[],"[32042, 98472, 475222, 56796, 662086, 666787, ...",[],"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,[],[],"[264320, 605720]",[],"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...,...,...,...,...
4293,120081,[],[],"[224607, 672096, 174950, 202906, 562039, 37070...",[],"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,[],[],"[260730, 243178, 19411, 311420, 55791, 406782,...",[],"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,[],[],"[67655, 65672, 210647, 608386, 413837, 566468,...",[],"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,[],[],"[145292, 402779, 452316, 324334, 217189, 42301...",[],"[가요리믹스, 김경호, 지아]"


In [166]:
not_tag_list_ida = [list(filter(lambda x: x in list(tag_embedding_model.wv.vocab.keys()), tag_addition['predicted_tags'][i])) for i in range(len(tag_addition['predicted_tags']))]

In [ ]:
list_for_list = []
test_predicted_tag = []
for i in range(len(not_tag_list_ida)):
  if len(not_tag_list_ida[i]) == 0:
    test_predicted_tag.append([])
  else:
    for k in range(10):
      list_for_list.append(tag_embedding_model.wv.most_similar(positive=not_tag_list_ida[i])[k][0])
    test_predicted_tag.append(list_for_list)
    print(i)
    list_for_list = []

In [181]:
tag_addition['additional_tag']= test_predicted_tag
tag_addition

,id,ply_token,tags,songs,predicted_songs,predicted_tags,additional_tag
0,70107,[],[],"[398985, 449403, 411543, 528044, 143048, 98020...",[],"[황현, 프리스타일, Apink, 고등학생]","[서지음, 오늘의추천가수, 슴스테, Monotree, 잼팩토리, 모노트리, eone..."
1,102395,[],[],"[528550, 285114, 506667, 339507, 653012, 435029]",[],"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep...","[UMFKOREA2017, 바운드, 울트라뮤직페스티벌, ElectroHouse, 매..."
2,42586,[],[],"[538867, 513110, 225168, 211527, 425074, 21709...",[],"[weeflow, kxan]","[Empire, 히팝, 제국, 센힙합, SMTM, nb2, 홍대클럽홍대엔비, 일리네..."
3,13244,[],[],"[32042, 98472, 475222, 56796, 662086, 666787, ...",[],"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피...","[VINXEN, 딥플로우, 사이먼도미닉, 키비, SIMONDOMINIC, SMTM,..."
4,55399,[],[],"[264320, 605720]",[],"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso...","[만화영화, 애니ost, 중음, 짱구, TV만화주제가, 그시절우리가좋아했던소녀, 쿵..."
...,...,...,...,...,...,...,...
4293,120081,[],[],"[224607, 672096, 174950, 202906, 562039, 37070...",[],"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW...","[본조비, 고스트스테이션, MMFD, ji, mymadfatdiary, ChuckB..."
4294,16489,[],[],"[260730, 243178, 19411, 311420, 55791, 406782,...",[],"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트...","[슴스테, 잼팩토리, 서지음, 방구석1열, EXO_CBX, 재현, 엔시티의나잇나잇,..."
4295,46470,[],[],"[67655, 65672, 210647, 608386, 413837, 566468,...",[],"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ...","[빈자리, 후폭풍, 가슴아픈노래, 이별후유증, 아프다, 리수, 우울할, 돋보이는, ..."
4296,48788,[],[],"[145292, 402779, 452316, 324334, 217189, 42301...",[],"[가요리믹스, 김경호, 지아]","[플라이투더스카이, MCTHEMAX, 노래방인기곡, 이별후에_듣는노래, 김종국, 슬..."


In [182]:
for i in range(len(tag_addition['predicted_tags'])):
  a = len(tag_addition['predicted_tags'][i])
  if a != 10:
    b = tag_addition['additional_tag'][i][0:10-a]
    tag_addition['predicted_tags'][i] = tag_addition['predicted_tags'][i] + b

In [186]:
a = tag_addition['id']
b = tag_addition['predicted_tags']
c = pd.concat([a,b], axis=1)
c.to_json('train2vec_addition.json')

In [237]:
c

,id,predicted_tags
0,70107,"[황현, 프리스타일, Apink, 고등학생, 서지음, 오늘의추천가수, 슴스테, Mo..."
1,102395,"[움프, 하드스타일, futurehouse, MartinGarrix, RunDeep..."
2,42586,"[weeflow, kxan, Empire, 히팝, 제국, 센힙합, SMTM, nb2..."
3,13244,"[힙발라드, 에픽하이, 드렁큰타이거, 재지팩트, 화지, Beenzino, 허클베리피..."
4,55399,"[JPOP모음, 중국노래, 중국음악, 일본애니, JAPAN, 일본밴드, chanso..."
...,...,...
4293,120081,"[잔잔한락, 추억의락, ACDC, 틴스피릿, Grunge, Metallica, NW..."
4294,16489,"[Kenzie, 폐막식, 켄지, 첸백시, 스엠, 엑소엘, 에프엑스, SM엔터테인먼트..."
4295,46470,"[생각이, 생각을, 아픈, 자기전이나, 감정이입, 슬퍼서, 우울함은, 감성발라드, ..."
4296,48788,"[가요리믹스, 김경호, 지아, 플라이투더스카이, MCTHEMAX, 노래방인기곡, 이..."
